# This code no longer works because the html of classic.com was changed on Tuesday, April 25, 2023. With that said, this code is how I captured all of the considered data in my model 

# Adding sold listings


In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import trange

# Iterate over each page of results
urls=[]
mileage_urls = []
comps_urls = []

for page in trange(1):
    url=f"https://www.classic.com/search?filters[driverside]=15&filters[location][]=US&filters[make]=7&filters[model]=13&filters[transmission]=17&q=defender&page={page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    all_data = soup.find_all('div', {'class': 'container'})
    all_data = all_data[1]
#     gen_data = soup.find('div', {'class': 'md:hidden mt-4'})
    gen_data = all_data.find('div', {'class': 'md:bg-white md:px-3 mb-3 md:shadow md:shadow-gray-400 md:rounded border-none md:border'})
    print(all_data)
    car_blocks = soup.find_all('div',{'class':'card bg-white flex-1'})
#     car_blocks = soup.find_all('div',{'class':'grid-item xs-12 sm-12 md-4 lg-3 relative'})
    for block in car_blocks:
        url = block.find('a')['href']
        main_url = "https://www.classic.com"+url[:-1]+'?tab=specs'
        mileage_url = "https://www.classic.com"+url[:-1]+'?tab=history'
        comps_url = "https://www.classic.com"+url[:-1]+'?tab=comps'
        urls.append(main_url)
        mileage_urls.append(mileage_url)
        comps_urls.append(comps_url)

100%|██████████| 1/1 [00:00<00:00,  3.41it/s]

<div class="container">
<div>
<div class="modal-wrapper hidden" phx-key="Escape" phx-window-keydown='[["push",{"event":"close_modal","target":2}]]'>
<div class="modal-backdrop" phx-click='[["push",{"event":"close_modal","target":2}]]'></div>
<div class="modal w-full max-w-full max-h-full sm:max-w-[448px] sm:max-h-[80%] sm:h-auto h-full" id="auth_modal">
<i class="material-icons close text-black" phx-click='[["push",{"event":"close_modal","target":2}]]'>close</i>
<div class="modal-title">
<span class="typography-h6 typography-primary-color typography-sm- typography-md- typography-lg- mt-8 block text-center">
        Welcome to CLASSIC.COM
      </span>
</div>
<div class="modal-content">
<div class="flex flex-col justify-center items-center px-6">
<div class="w-full">
<div>
<ul class="flex justify-around text-base mb-4">
<li class="p-2 cursor-pointer is-active border-black border-solid border-b-2 flex-1 text-center" phx-click='[["push",{"event":"tab_click","target":9}]]' phx-value-index=

In [15]:
import re
from tqdm import trange
data=[]
titles=[]
model_types=[]
standard_pattern = r"(90|110|130)\s*(.+)?"
for i in trange(3):
    url = urls[i]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    specs_tab = soup.find('div', {'class': 'card bg-white grid grid-flow-row md:grid-flow-col p-4 md:p-8'})
    block3 = specs_tab.find('div', {'class': 'grid-item xs-12 md-3 lg-3'})
    main_block = soup.find('div', {'class': 'grid-item xs-12 md-6 p-4 md:p-8'})
    price = main_block.find('div', {'class': 'space-y-1 mt-4'})
    status_rough = price.find_all('h3',{'class':'flex text-base items-center'})
    status_rough = status_rough[0]
    try:
        status_final = status_rough.find('span',{'class':'font-medium'}).text.strip()
    except:
        status_final = status_rough.find('span',{'class':'flex-1'})
        status_final = status_final.find('span').text.strip()
    if 'sold' in status_final.lower():
        ext_color = block3.find('div', {'class':"py-4 space-y-2"})
        ext_color_draft = ext_color.find_all('div', {'class':"xs-6 sm-6 md-5 lg-4 font-medium"})
        ext_color_final = ext_color_draft[2].text.strip()
        block1 = specs_tab.find('div', {'class': 'grid-item xs-12 md-4 lg-4 border-b-[0.5px] border-gray-300 md:border-0'})
        model_variant = block1.find('div', {'class':"py-4 space-y-2"})
        model_variant_draft = model_variant.find_all('div', {'class':"xs-6 sm-6 md-5 lg-4 font-medium"})
        model_variant_final = model_variant_draft[2].text.strip()
        title = soup.find('h1',{'class':'text-2xl'}).text.strip()
        titles.append(title)

        year = int(title[:4])
        age = 2023-year
        match = re.search(standard_pattern, title)
        if match:
            model_type = match.group(2)
            if not model_type:
                model_type = 'standard'
        model_type.lower()
    #     price = main_block.find('div', {'class': 'space-y-1 mt-4'})
        price_draft = price.find('h3', {'class':'flex text-base items-center'})
        price_element = price_draft.find('div',{'class':'text-green-700 font-medium inline-block'})
        price_final = price_element.text.strip() if price_element else ''
        price_final = int(re.sub(r'\D', '', price_final)) if price_final else 0

        mileage = price.find_all('h3',{'class':'flex text-base items-center'})
        mileage_draft = mileage[2]
        mileage_element = mileage_draft.find('span',{'class':'flex-1'})
        mileage_text = mileage_element.text.strip()
        if 'kilometers' in mileage_text.lower():
            i = mileage_text.find('(')
            mileage_final = int(re.sub(r'[^\d.]', '', mileage_text[:i]))
            mileage_final *= 0.621371
        elif 'miles' not in mileage_text.lower():
            mileage_final = 0
        else:
            mileage_final = int(re.sub(r'[^\d.]', '', mileage_text))
        try:
            originality_rough = mileage[5]
        except IndexError:
            try:
                originality_rough = mileage[4]
            except IndexError:
                originality_rough = mileage[3]
        try:
            originality_final = originality_rough.find('div',{'class':'flex flex-row w-full'})
            originality_final = originality_final.find('span')
            originality_final = originality_final.text.strip()
        except:
            originality_final = 'Restored-Modified'
    else:
        ext_color = block3.find('div', {'class':"py-4 space-y-2"})
        ext_color_draft = ext_color.find_all('div', {'class':"xs-6 sm-6 md-5 lg-4 font-medium"})
        ext_color_final = ext_color_draft[2].text.strip()
        block1 = specs_tab.find('div', {'class': 'grid-item xs-12 md-4 lg-4 border-b-[0.5px] border-gray-300 md:border-0'})
        model_variant = block1.find('div', {'class':"py-4 space-y-2"})
        model_variant_draft = model_variant.find_all('div', {'class':"xs-6 sm-6 md-5 lg-4 font-medium"})
        model_variant_final = model_variant_draft[2].text.strip()
        main_block = soup.find('div', {'class': 'grid-item xs-12 md-6 p-4 md:p-8'})
        title = soup.find('h1',{'class':'text-2xl'}).text.strip()
        titles.append(title)
        year = int(title[:4])
        age = 2023-year
        match = re.search(standard_pattern, title)
        if match:
            model_type = match.group(2)
            if not model_type:
                model_type = 'standard'
        model_type.lower()
        price = main_block.find('div', {'class': 'space-y-1 mt-4'})
        price_draft = price.find('h3', {'class':'flex text-base items-center'})
        price_element = price_draft.find('div',{'class':'text-black inline-block'})
        price_final = price_element.text.strip() if price_element else ''
        try:
            price_final = int(re.sub(r'\D', '', price_final))
        except ValueError:
            price_final = 'comps'   
        mileage = price.find_all('h3',{'class':'flex text-base items-center'})
        mileage_draft = mileage[2]
        mileage_element = mileage_draft.find('span',{'class':'flex-1'})
        mileage_text = mileage_element.text.strip()
        if 'kilometers' in mileage_text.lower():
            i = mileage_text.find('(')
            mileage_final = int(re.sub(r'[^\d.]', '', mileage_text[:i]))
            mileage_final *= 0.621371
        elif 'miles' not in mileage_text.lower():
            mileage_final = 0
        else:
            mileage_final = int(re.sub(r'[^\d.]', '', mileage_text))
        try:
            originality_rough = mileage[5]
        except IndexError:
            try:
                originality_rough = mileage[4]
            except IndexError:
                originality_rough = mileage[3]
        try:
            originality_final = originality_rough.find('div',{'class':'flex flex-row w-full'})
            originality_final = originality_final.find('span')
            originality_final = originality_final.text.strip()
        except:
            originality_final = 'Restored-Modified'
        
    data.append([price_final,model_type,model_variant_final,ext_color_final,mileage_final,age,originality_final])

  0%|          | 0/3 [00:01<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'find'

In [7]:
price_final=''
price_final = int(re.sub(r'\D', '', price_final))
price_final

ValueError: invalid literal for int() with base 10: ''

In [6]:
df = pd.DataFrame(data, columns=['price_final','model_type','model_variant_final','ext_color_final','mileage_final','age','originality_final'])
# df = df[df['price_final'] != 0]
df['price_final'].value_counts()

0         81
55000     12
35000     10
75000      8
comps      8
          ..
130876     1
31995      1
67900      1
40250      1
181000     1
Name: price_final, Length: 222, dtype: int64

# Verifying/Replacing Mileage

In [19]:
import re
from tqdm import trange
mileage_data=[]

for i in trange(60):
    url = mileage_urls[i]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    price = soup.find('div', {'class': 'card bg-white p-0 mb-0'})
    mileage_block = price.find('div',{'class':'flex flex-col flex-1 justify-center py-4'})
    try:
        mileage_rough = mileage_block.find_all('span',{'class':'text-gray-500 mr-1'})[1]
        mileage_text = mileage_rough.text.strip()
        if 'kilometers' in mileage_text.lower():
            print(mileage_text)
            i = mileage_text.find('(')
            mileage_final = int(re.sub(r'[^\d.]', '', mileage_text[:i]))
            mileage_final *= 0.621371
            mileage_final = int(mileage_final)
        elif 'miles' not in mileage_text.lower():
            mileage_rough = mileage_block.find_all('span',{'class':'text-gray-500 mr-1'})
            try:
                mileage_rough = mileage_rough[0]
                mileage_text = mileage_rough.text.strip()
                if 'kilometers' in mileage_text.lower():
                    i = mileage_text.find('(')
                    mileage_final = int(re.sub(r'[^\d.]', '', mileage_text[:i]))
                    mileage_final *= 0.621371
                    mileage_final = int(mileage_final)
                elif 'miles' in mileage_text.lower():
                    mileage_final = int(re.sub(r'[^\d.]', '', mileage_text))
                else:
                    mileage_final = 'delete'
            except IndexError:
                mileage_final = 'delete'
        else:
            mileage_final = int(re.sub(r'[^\d.]', '', mileage_text))
    except IndexError:
        mileage_final = 'delete'
    mileage_data.append(mileage_final)

  2%|▏         | 1/60 [00:03<03:55,  3.99s/it]

63,000 miles   ·
63000



  3%|▎         | 2/60 [00:06<03:19,  3.44s/it]

156,000 kilometers (96,934 miles) (TMU) ·
96933



  5%|▌         | 3/60 [00:08<03:05,  3.25s/it]

...

225,000 kilometers (139,808 miles)  ·
139808



 98%|█████████▊| 59/60 [02:27<00:03,  3.00s/it]

163,000 kilometers (101,283 miles)  ·
101283


100%|██████████| 60/60 [02:30<00:00,  2.50s/it]

77,577 miles   ·
77577


In [184]:
df_mileage = pd.DataFrame(mileage_data, columns=['mileage_data'])
df_mileage.value_counts()

mileage_data
delete          54
208000129245     6
3000             5
98000            4
5500034175       4
                ..
162657           1
164560           1
165000           1
166000           1
108600           1
Length: 351, dtype: int64

In [20]:
# count=0
# for i, miles in enumerate(df_mileage['mileage_data']):
#     if miles!='delete':
#         miles = float(miles)
#         if df.at[i,'mileage_final']!=df_mileage.at[i,'mileage_data']:
#             print(df.at[i,'mileage_final'], miles)
# #     if len(str(miles))==7:
# #         miles = str(miles)
# #         miles = int(miles[4:]) 
# #         df_mileage.at[i,'mileage_data'] = miles
# #     elif len(str(miles))==8:
# #         count+=1
# #         miles = str(miles)
# #         miles = int(miles[4:])
# #         df_mileage.at[i,'mileage_data'] = miles
# #     elif len(str(miles))==9:
# #         count+=1
# #         miles = str(miles)
# #         miles = int(miles[5:])
# #         df_mileage.at[i,'mileage_data'] = miles
# #     elif len(str(miles))==10:
# #         count+=1
# #         miles = str(miles)
# #         miles = int(miles[5:])
# #         df_mileage.at[i,'mileage_data'] = miles
# #     elif len(str(miles))==12:
# #         count+=1
# #         miles = str(miles)
# #         try:
# #             miles = int(miles[6:])
# #         except:
# #             print(i,miles)
# count

In [123]:
df['mileage_final'].value_counts()

0.000         206
129245.168      6
98000.000       4
114332.264      3
173362.509      3
             ... 
174227.000      1
115400.000      1
118681.861      1
108600.000      1
69950.000       1
Name: mileage_final, Length: 251, dtype: int64

In [122]:
df = df.reset_index(drop=True)
df_mileage = df_mileage.reset_index(drop=True)

In [124]:
# loop through the rows of both data frames
for i,mileage in enumerate(df['mileage_final']):   
    mileage = str(mileage)
    if mileage == '0.0':
        if df_mileage.at[i,'mileage_data']=='delete':
            df_mileage.at[i,'mileage_data'] = 0
        else:
            df.at[i, 'mileage_final'] = df_mileage.at[i, 'mileage_data']           

In [125]:
df['mileage_final'].value_counts()

0.000000         54
129245.168000     6
3000.000000       5
98000.000000      4
983712.000000     3
                 ..
44000.000000      1
109361.296000     1
93336.759281      1
73343.000000      1
11806.049000      1
Name: mileage_final, Length: 357, dtype: int64

In [126]:
df = df.reset_index(drop=True)

# Adding Price to na price_final values using comp>90% 

In [72]:
import re
from tqdm import trange
comp_price_data=[]

for i in trange(len(comps_urls)-1):
    url = comps_urls[i]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    main_block = soup.find('div', {'class': 'grid-item xs-12 md-6 p-4 md:p-8'})

    price = soup.find('div', {'class': 'card bg-white p-0 mb-0'})
    price = main_block.find('div', {'class': 'space-y-1 mt-4'})
    price_draft = price.find('h3', {'class':'flex text-base items-center'})
    price_element = price_draft.find('div',{'class':'text-black inline-block'})
    price_final = price_element.text.strip() if price_element else ''
    try:
        price_final = int(re.sub(r'\D', '', price_final))
    except ValueError:
        price_final = 'comps'
    if price_final == 'comps':
        big_block = soup.find('div',{'class':'flex flex-col divide-y divide-gray-200'})
        comps_block = big_block.find_all('div', {'class':'duration-1000 px-2 py-2'})
        comps_block = comps_block[0]
        relevance_score = comps_block.find('div',{'class':'flex flex-row items-center justify-right md:hidden'})
        relevance_score_draft = relevance_score.find('div',{'class':'shadow-none flex flex-col text-center whitespace-nowrap text-white justify-center bg-gray-800'})
        relevance_score_final = relevance_score_draft.get('style')
        relevance_score_final = float(relevance_score_final.split(':')[1].strip('%'))
        if relevance_score_final>90:
            comps_price = big_block.find('div',{'class':'flex flex-col xs-8 justify-center flex-1 ml-4'})
            comps_price_draft = comps_price.find('div',{'class':'text-black inline-block'})
            comps_price_final = comps_price.find('div',{'class':'text-green-700 font-medium inline-block'})
            if comps_price_final!=None:
                price_final = float((comps_price_final.text.strip()).replace(',', '').replace('$', ''))
            else:
                try:
                    comps_price_final = comps_price_draft.find_all('span')[1]
                    comps_price_final = comps_price_final.text.strip().replace(',', '').replace('$', '').replace('.','').strip()
                    price_final = float(comps_price_final[8:])
                except:
                    price_final = 0
                    print(url)
        else:
            price_final = 0
    comp_price_data.append(price_final)

 12%|█▏        | 61/524 [02:20<17:10,  2.22s/it]

https://www.classic.com/veh/2023-land-rover-defender-130-first-edition-salevfe-n2PAwop?tab=comps


 40%|████      | 210/524 [08:19<13:32,  2.59s/it]

https://www.classic.com/veh/1991-land-rover-defender-110-salldhmd8ff801486-WxjNwa4?tab=comps


 49%|████▉     | 259/524 [10:23<10:42,  2.43s/it]

...

 90%|████████▉ | 471/524 [18:58<02:08,  2.42s/it]

https://www.classic.com/veh/1987-land-rover-defender-90-salldvab8aa284531-WkPZRlp?tab=comps


 98%|█████████▊| 514/524 [20:36<00:21,  2.14s/it]

https://www.classic.com/veh/1994-land-rover-defender-90-nas-5-speed-saldv2282r-nB1rJ94?tab=comps


100%|██████████| 524/524 [21:00<00:00,  2.41s/it]


In [127]:
df_comp_price = pd.DataFrame(comp_price_data, columns=['comp_price_data'])
df_comp_price

,comp_price_data
0,49990.0
1,91500.0
2,88995.0
3,57999.0
4,112000.0
...,...
519,53000.0
520,50600.0
521,60480.0
522,65000.0


In [136]:
for i,price in enumerate(df['price_final']):
    df.at[i,'price_final'] = df_comp_price.at[i,'comp_price_data']

In [138]:
df['price_final'].value_counts()

0.0         38
55000.0     12
35000.0     10
66000.0      9
73000.0      8
            ..
187000.0     1
128800.0     1
36100.0      1
47100.0      1
132222.0     1
Name: price_final, Length: 224, dtype: int64

In [139]:
df = df.reset_index(drop=True)

In [140]:
df

,price_final,model_type,model_variant_final,ext_color_final,mileage_final,age,originality_final
0,49990,Land Rover Defender 90,90 Hard Top,Gray,0.000000e+00,33,Restored-Modified
1,91500,NAS 5-Speed,90 Soft Top,Green,6.300000e+04,28,Original & Highly Original
2,88995,5-Speed,110 Hard Top,Blue,1.560010e+10,31,Restored-Modified
3,57999,300TDI 5-Speed,110 Wagon,Blue,1.733625e+05,27,Restored-Original
4,112000,5-Speed,90 Soft Top,Black,1.030000e+05,29,Restored-Original
...,...,...,...,...,...,...,...
519,53000,2.0L Mpi,90 Soft Top,Blue,1.323520e+05,28,Restomod & Custom
520,50600,Country Station Wagon,110 Wagon,Green,5.355800e+04,40,Restored-Modified
521,60480,NAS,90 Soft Top,Yellow,0.000000e+00,29,Original & Highly Original
522,65000,Land Rover Defender 110,110 Wagon,White,3.417540e+04,33,Restored


In [146]:
df = df.drop(df[(df['price_final'] == 0) & (df['mileage_final'] == 0)].index)

In [147]:
df = df.drop(df.loc[df['price_final'] == 0].index)

In [148]:
df = df.reset_index(drop=True)

In [22]:
for i,mile in enumerate(df['mileage_final']):
    if mile>300000:
        print(mile)

NameError: name 'df' is not defined

In [149]:
df

,price_final,model_type,model_variant_final,ext_color_final,mileage_final,age,originality_final
0,49990,Land Rover Defender 90,90 Hard Top,Gray,0.000000e+00,33,Restored-Modified
1,91500,NAS 5-Speed,90 Soft Top,Green,6.300000e+04,28,Original & Highly Original
2,88995,5-Speed,110 Hard Top,Blue,1.560010e+10,31,Restored-Modified
3,57999,300TDI 5-Speed,110 Wagon,Blue,1.733625e+05,27,Restored-Original
4,112000,5-Speed,90 Soft Top,Black,1.030000e+05,29,Restored-Original
...,...,...,...,...,...,...,...
481,53000,2.0L Mpi,90 Soft Top,Blue,1.323520e+05,28,Restomod & Custom
482,50600,Country Station Wagon,110 Wagon,Green,5.355800e+04,40,Restored-Modified
483,60480,NAS,90 Soft Top,Yellow,0.000000e+00,29,Original & Highly Original
484,65000,Land Rover Defender 110,110 Wagon,White,3.417540e+04,33,Restored


In [150]:
df.to_csv('updated_classic_defender.csv', index=False)